----

---

# Análisis de Base de Datos - SQL

## Objetivo del Estudio

El propósito de este análisis es explorar una base de datos relacionada con libros, editoriales, autores y calificaciones de clientes.  
Usaremos consultas SQL para obtener información relevante que nos ayude a entender el comportamiento de los lectores y editoriales.  

Se realizarán análisis para:  
1. Determinar cuántos libros fueron publicados después del **1 de enero de 2000**.  
2. Obtener el número de **reseñas de usuarios** y la **calificación promedio** de cada libro.  
3. Identificar la **editorial con más publicaciones** de libros con más de 50 páginas.  
4. Encontrar el **autor con la más alta calificación promedio**, considerando solo libros con al menos 50 calificaciones.  
5. Calcular el **número promedio de reseñas de texto** entre los usuarios que han calificado más de 50 libros.  


##  Plan de Trabajo

###  Inicialización  

In [6]:
# Importar las librerías necesarias.

import pandas as pd 
from sqlalchemy import create_engine


# Conectarse a la base de datos siguiendo las instrucciones proporcionadas

db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Exploración de Datos 

In [7]:
# Consultar las primeras filas de cada tabla para entender sus relaciones

tables = ["books", "authors", "publishers", "ratings", "reviews"]

# Obtener las primeras 10 filas de cada una

for table in tables:
    query = f"SELECT * FROM {table} LIMIT 5"
    df = pd.io.sql.read_sql(query, con=engine)
    
    print(f"\n Primeras filas de la tabla {table}:")
    print(df)




 Primeras filas de la tabla books:
   book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125                                               1776   

   num_pages publication_date  publisher_id  
0        594       2005-11-01            93  
1        992       2003-05-22           336  
2        322       2010-12-21           135  
3        541       2006-10-10           309  
4        386       2006-07-04           268  

 Primeras filas de la tabla authors:
   author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4 

### Análisis de Datos con SQL


#### Consulta 1: Número de libros publicados después del 1 de enero de 2000

In [8]:
# Consulta 

consulta_1 = """
SELECT COUNT(*) AS libros_publicados
FROM books
WHERE publication_date > '2000-01-01'
"""

# Ejecutar la consulta 

consulta_uno = pd.io.sql.read_sql(consulta_1, con=engine)
print(consulta_uno)


   libros_publicados
0                819


**Notas:**  819 libros publicados después del 1 de enero de 2000. 

#### Consulta 2: Número de reseñas de usuarios y la calificación promedio para cada libro

In [9]:
# Consulta

consulta_2 = """
SELECT 
    books.book_id,
    books.title,
    COUNT(reviews.review_id) AS num_resenas,
    AVG(ratings.rating) AS calificacion_promedio
FROM books
LEFT JOIN reviews ON books.book_id = reviews.book_id
LEFT JOIN ratings ON books.book_id = ratings.book_id
GROUP BY books.book_id, books.title
ORDER BY num_resenas DESC;

"""

# Ejecutar la consulta 

df_resenas_calificaciones = pd.io.sql.read_sql(consulta_2, con=engine)
print(df_resenas_calificaciones)


     book_id                                              title  num_resenas  \
0        948                            Twilight (Twilight  #1)         1120   
1        750                The Hobbit  or There and Back Again          528   
2        673                             The Catcher in the Rye          516   
3        302  Harry Potter and the Prisoner of Azkaban (Harr...          492   
4        299  Harry Potter and the Chamber of Secrets (Harry...          480   
..       ...                                                ...          ...   
995      221                          Essential Tales and Poems            0   
996      808                            The Natural Way to Draw            0   
997      191  Disney's Beauty and the Beast (A Little Golden...            0   
998       83    Anne Rice's The Vampire Lestat: A Graphic Novel            0   
999      672   The Cat in the Hat and Other Dr. Seuss Favorites            0   

     calificacion_promedio  
0         

**Notas:** El análisis de las reseñas y calificaciones de los libros muestra que algunos títulos, como Twilight y The Hobbit, son ampliamente comentados con más de 500 reseñas, mientras que otros no tienen ninguna. La calificación promedio varía, con algunos libros alcanzando valores altos de 4.5 o más, mientras que otros tienen puntuaciones más bajas. Esto sugiere que la popularidad de un libro (medida por el número de reseñas) no siempre está directamente relacionada con su calificación promedio.

#### Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas 

In [10]:
# Consulta 

consulta_3 = """
SELECT 
    publishers.publisher_id,
    publishers.publisher AS publisher_name,
    COUNT(books.book_id) AS num_libros
FROM books
JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher_id, publishers.publisher
ORDER BY num_libros DESC
LIMIT 1;
"""

# Ejecutar la consulta 

df_editorial_top = pd.io.sql.read_sql(consulta_3, con=engine)
print(df_editorial_top)


   publisher_id publisher_name  num_libros
0           212  Penguin Books          42


**Notas:** La editorial Penguin Books es la que ha publicado la mayor cantidad de libros con más de 50 páginas, con un total de 42 libros.

#### Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones

In [11]:
# Consulta 

consulta_4 = """
SELECT 
    authors.author_id,
    authors.author,
    AVG(ratings.rating) AS calificacion_promedio
FROM books
JOIN authors ON books.author_id = authors.author_id
JOIN ratings ON books.book_id = ratings.book_id
GROUP BY authors.author_id, authors.author
HAVING COUNT(ratings.rating) >= 50
ORDER BY calificacion_promedio DESC
LIMIT 1;
"""

# Ejecutar la consulta 

df_autor_top = pd.io.sql.read_sql(consulta_4, con=engine)
print(df_autor_top)
    
    

   author_id          author  calificacion_promedio
0        130  Diana Gabaldon                    4.3


**Notas:**
La autora con la calificación promedio más alta, considerando solo los libros con al menos 50 calificaciones, es Diana Gabaldon, con una calificación promedio de 4.3.

#### Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros

In [12]:
# Consulta

consulta_5 = """
    WITH usuarios_con_mas_de_50_calificaciones AS (
        SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(rating_id) > 50
    )
    SELECT AVG(num_resenas) AS promedio_resenas
    FROM (
        SELECT COUNT(r.review_id) AS num_resenas
        FROM reviews r
        JOIN usuarios_con_mas_de_50_calificaciones u ON r.username = u.username
        GROUP BY r.username
    ) AS subquery;
"""
# Ejecutar la consulta

promedio_resenas_df = pd.io.sql.read_sql(consulta_5, con=engine)
print(promedio_resenas_df)


   promedio_resenas
0         24.333333


**Notas:** El análisis muestra que, en promedio, los usuarios que han calificado más de 50 libros han escrito alrededor de 24 reseñas de texto. Esto sugiere que aunque estos usuarios son activos calificadores, no siempre acompañan sus calificaciones con comentarios detallados.

## Conclusión Final 

El análisis de las consultas realizadas proporciona información valiosa sobre la dinámica de calificaciones, reseñas y publicaciones en la base de datos de libros:

- Número de libros publicados después del 1 de enero de 2000: Se identificó que 819 libros han sido publicados después de esta fecha, lo que indica una presencia significativa de literatura reciente en la base de datos.


- Número de reseñas de usuarios y la calificación promedio por libro: Se encontró que algunos libros, como Twilight y The Hobbit, tienen un alto número de reseñas, pero la cantidad de reseñas no siempre está correlacionada con la calificación promedio, lo que sugiere diferencias en la percepción de calidad entre los lectores.


- Editorial con mayor número de libros con más de 50 páginas: Penguin Books es la editorial con más publicaciones de este tipo, con 42 libros, lo que resalta su liderazgo en la industria con una amplia gama de contenido sustancial.


- Autor con la calificación promedio más alta (considerando libros con al menos 50 calificaciones): Diana Gabaldon, con una calificación promedio de 4.3, se posiciona como la autora mejor valorada, lo que refleja la alta satisfacción de los lectores con sus obras.


- Promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros: Se determinó que estos usuarios han escrito en promedio 24 reseñas de texto, lo que sugiere que, aunque son activos calificadores, muchos prefieren expresar su opinión solo con puntuaciones en lugar de comentarios detallados.